In [1]:
#coded by shibinmak in amazon EC2 environment.

In [2]:
from pyspark import SparkConf,SparkContext

In [3]:
import pandas as pd

In [4]:
def loadNames():
    movie_details={}
    data=pd.read_csv('movies-Copy1.csv')
    movie_id =list(data['movieId'])
    movie_name=list(data['title'])
    for i in range(len(movie_id)):
        movie_details[movie_id[i]] = movie_name[i]
    
    return movie_details
    
    

In [5]:
conf =SparkConf().setMaster('local').setAppName('popular movie')
    

In [6]:
sc =SparkContext(conf=conf)
    

In [7]:
namedict=sc.broadcast(loadNames())

In [8]:
data=pd.read_csv('ratings-Copy1.csv',header=None)

In [9]:
data.drop(0,axis=0,inplace=True)

In [10]:
data.head()

,0,1,2,3
1,1,31,2.5,1260759144
2,1,1029,3.0,1260759179
3,1,1061,3.0,1260759182
4,1,1129,2.0,1260759185
5,1,1172,4.0,1260759205


In [11]:
data.to_csv('ratings.csv',index=None,header=None)

In [12]:
rdd1 = sc.textFile('ratings.csv')

In [13]:
rdd2 = rdd1.map(lambda x:(int(x.split(',')[1]),1))

In [ ]:
#rdd2.collect()

In [15]:
movie_id = rdd2.reduceByKey(lambda x,y:x+y)

In [ ]:
#movie_id.collect()

In [17]:
swap = movie_id.map(lambda x:(x[1],x[0]))

In [18]:
sortedresult = swap.sortByKey()

In [19]:
moviewithname = sortedresult.map(lambda x:(namedict.value[x[1]],x[0]))

In [20]:
results=moviewithname.collect()

In [ ]:
for x in results:
    print("{}  :   {}".format(x[0],x[1]))